In [ ]:
import mysql.connector
from faker import Faker
import random

# Configuration
HOST = "localhost"
PORT = 33060
USER = "root"
PASSWORD = "IU2026"
DATABASE = "webshop"

def connect_to_db(with_db=False):
    """Connect to MariaDB instance"""
    config = {
        "host": HOST,
        "port": PORT,
        "user": USER,
        "password": PASSWORD,
    }
    if with_db:
        config["database"] = DATABASE

    return mysql.connector.connect(**config)

def create_database_and_tables():
    """Create the webshop database and tables"""
    conn = connect_to_db()
    cursor = conn.cursor()

    try:
        # Drop database if it exists (for clean slate)
        print("Dropping existing database if it exists...")
        cursor.execute(f"DROP DATABASE IF EXISTS {DATABASE}")

        # Create database
        print(f"Creating database '{DATABASE}'...")
        cursor.execute(f"CREATE DATABASE {DATABASE}")
        conn.commit()

        # Select database
        cursor.execute(f"USE {DATABASE}")

        # Create customers table
        print("Creating customers table...")
        cursor.execute("""
            CREATE TABLE customers (
                customer_id INT AUTO_INCREMENT PRIMARY KEY,
                first_name VARCHAR(100) NOT NULL,
                last_name VARCHAR(100) NOT NULL,
                email VARCHAR(150) UNIQUE NOT NULL,
                phone VARCHAR(20),
                address VARCHAR(255),
                city VARCHAR(100),
                postal_code VARCHAR(20),
                country VARCHAR(100),
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)

        # Create products table
        print("Creating products table...")
        cursor.execute("""
            CREATE TABLE products (
                product_id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(255) NOT NULL,
                description TEXT,
                price DECIMAL(10, 2) NOT NULL,
                category VARCHAR(100),
                stock_quantity INT DEFAULT 0,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)

        # Create orders table
        print("Creating orders table...")
        cursor.execute("""
            CREATE TABLE orders (
                order_id INT AUTO_INCREMENT PRIMARY KEY,
                customer_id INT NOT NULL,
                order_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                total_amount DECIMAL(12, 2),
                status ENUM('pending', 'processing', 'shipped', 'delivered', 'cancelled') DEFAULT 'pending',
                FOREIGN KEY (customer_id) REFERENCES customers(customer_id) ON DELETE CASCADE
            )
        """)
        # Create order_products table (junction table for orders and products)
        print("Creating order_products table...")
        cursor.execute("""
            CREATE TABLE order_products (
          order_product_id INT AUTO_INCREMENT PRIMARY KEY,
          order_id INT NOT NULL,
          product_id INT NOT NULL,
          quantity INT NOT NULL DEFAULT 1,
          price_at_order DECIMAL(10, 2) NOT NULL,
          FOREIGN KEY (order_id) REFERENCES orders(order_id) ON DELETE CASCADE,
          FOREIGN KEY (product_id) REFERENCES products(product_id) ON DELETE CASCADE
            )
        """)
        conn.commit()
        print("Database and tables created successfully!")

    finally:
        cursor.close()
        conn.close()

def populate_customers(num_customers=10000):
    """Generate and insert synthetic customer data"""
    conn = connect_to_db(with_db=True)
    cursor = conn.cursor()
    fake = Faker()

    try:
        print(f"Generating and inserting {num_customers} customers...")

        customers_data = []
        seen_emails = set()

        for i in range(num_customers):
            # Generate unique email
            email = fake.unique.email()
            while email in seen_emails:
                email = fake.email()
            seen_emails.add(email)

            customer = (
                fake.first_name(),
                fake.last_name(),
                email,
                fake.phone_number()[:20],
                fake.street_address(),
                fake.city(),
                fake.postcode(),
                fake.country()
            )
            customers_data.append(customer)

            # Batch insert every 1000 rows for efficiency
            if (i + 1) % 1000 == 0:
                cursor.executemany(
                    """INSERT INTO customers
                       (first_name, last_name, email, phone, address, city, postal_code, country)
                       VALUES (%s, %s, %s, %s, %s, %s, %s, %s)""",
                    customers_data
                )
                conn.commit()
                print(f"  Inserted {i + 1} customers...")
                customers_data = []

        # Insert remaining rows
        if customers_data:
            cursor.executemany(
                """INSERT INTO customers
                   (first_name, last_name, email, phone, address, city, postal_code, country)
                   VALUES (%s, %s, %s, %s, %s, %s, %s, %s)""",
                customers_data
            )
            conn.commit()

        print(f"Successfully inserted {num_customers} customers!")

    finally:
        cursor.close()
        conn.close()

def populate_products(num_products=10000):
    """Generate and insert synthetic product data"""
    conn = connect_to_db(with_db=True)
    cursor = conn.cursor()
    fake = Faker()

    categories = ["Electronics", "Clothing", "Home & Garden", "Sports", "Books",
                  "Toys", "Beauty", "Food & Beverages", "Furniture", "Automotive"]

    try:
        print(f"Generating and inserting {num_products} products...")

        products_data = []

        for i in range(num_products):
            product = (
                fake.word().capitalize() + " " + fake.word().capitalize(),
                fake.sentence(nb_words=8),
                round(random.uniform(5.99, 999.99), 2),
                random.choice(categories),
                random.randint(0, 1000)
            )
            products_data.append(product)

            # Batch insert every 1000 rows for efficiency
            if (i + 1) % 1000 == 0:
                cursor.executemany(
                    """INSERT INTO products
                       (name, description, price, category, stock_quantity)
                       VALUES (%s, %s, %s, %s, %s)""",
                    products_data
                )
                conn.commit()
                print(f"  Inserted {i + 1} products...")
                products_data = []

        # Insert remaining rows
        if products_data:
            cursor.executemany(
                """INSERT INTO products
                   (name, description, price, category, stock_quantity)
                   VALUES (%s, %s, %s, %s, %s)""",
                products_data
            )
            conn.commit()

        print(f"Successfully inserted {num_products} products!")

    finally:
        cursor.close()
        conn.close()

def verify_data():
    """Verify the inserted data"""
    conn = connect_to_db(with_db=True)
    cursor = conn.cursor()

    try:
        cursor.execute("SELECT COUNT(*) FROM customers")
        customer_count = cursor.fetchone()[0]

        cursor.execute("SELECT COUNT(*) FROM products")
        product_count = cursor.fetchone()[0]

        cursor.execute("SELECT COUNT(*) FROM orders")
        order_count = cursor.fetchone()[0]

        print("\n=== Data Summary ===")
        print(f"Total customers: {customer_count}")
        print(f"Total products: {product_count}")
        print(f"Total orders: {order_count}")

    finally:
        cursor.close()
        conn.close()


In [ ]:
# create db and tables
print("Starting webshop database setup...\n")

# Create database and tables
create_database_and_tables()

# Populate tables with synthetic data
populate_customers(num_customers=10000)
populate_products(num_products=10000)

# Verify data
verify_data()

print("Webshop database setup completed successfully!")

In [ ]:
import random
from datetime import datetime, timedelta
import time
from concurrent.futures import ThreadPoolExecutor

def fetch_customers(cursor):
    cursor.execute("SELECT customer_id FROM customers")
    return [row[0] for row in cursor.fetchall()]

def fetch_products(cursor):
    cursor.execute("SELECT product_id, price, stock_quantity FROM products")
    return [(row[0], float(row[1]), int(row[2])) for row in cursor.fetchall()]

def pick_cart_items(products, max_items=5, max_qty=3):
    available = [p for p in products if p[2] > 0]
    pool = available if available else products

    items_count = random.randint(1, max_items)
    chosen = random.sample(pool, k=items_count) if len(pool) >= items_count else random.choices(pool, k=items_count)

    items = []
    for product_id, price, stock_qty in chosen:
        qty_limit = min(max_qty, stock_qty) if stock_qty > 0 else max_qty
        quantity = random.randint(1, max(1, qty_limit))
        line_total = round(price * quantity, 2)
        items.append((product_id, quantity, price, line_total))

    return items

def create_order_with_items(conn, customer_id, items):
    """Create an order with items using an existing connection."""
    cursor = conn.cursor()
    start_time = time.perf_counter()
    try:
        conn.start_transaction()

        cursor.execute(
            """
            INSERT INTO orders (customer_id, total_amount, status)
            VALUES (%s, %s, %s)
            """,
            (customer_id, 0, "pending"),
        )
        order_id = cursor.lastrowid

        order_products = [
            (order_id, product_id, quantity, unit_price)
            for product_id, quantity, unit_price, _line_total in items
        ]
        cursor.executemany(
            """
            INSERT INTO order_products
                (order_id, product_id, quantity, price_at_order)
            VALUES (%s, %s, %s, %s)
            """,
            order_products,
        )

        total_amount = round(sum(item[3] for item in items), 2)
        cursor.execute(
            "UPDATE orders SET total_amount = %s WHERE order_id = %s",
            (total_amount, order_id),
        )

        # Subtract ordered items from stock_quantity
        for product_id, quantity, _unit_price, _line_total in items:
            cursor.execute(
          "UPDATE products SET stock_quantity = stock_quantity - %s WHERE product_id = %s",
          (quantity, product_id)
            )

        conn.commit()
        
        elapsed_ms = (time.perf_counter() - start_time) * 1000
        return order_id, elapsed_ms
    except Exception:
        conn.rollback()
        raise
    finally:
        cursor.close()

def advance_order_status(conn, order_id, status):
    cursor = conn.cursor()
    try:
        if conn.in_transaction:
            raise RuntimeError("Already in a transaction. Nested transactions are not supported.")

        conn.start_transaction()
        cursor.execute(
            "UPDATE orders SET status = %s WHERE order_id = %s",
            (status, order_id),
        )
        conn.commit()
    except Exception:
        conn.rollback()
        raise
    finally:
        cursor.close()

def simulate_shopping_carts(num_orders=200, max_items=5, max_qty=3, cancel_rate=0.1):
    conn = connect_to_db(with_db=True)

    cursor = conn.cursor()
    try:
        customers = fetch_customers(cursor)
        products = fetch_products(cursor)

        if not customers:
            raise RuntimeError("No customers available. Populate customers first.")
        if not products:
            raise RuntimeError("No products available. Populate products first.")

        if conn.in_transaction:
            conn.commit()

        cursor.close()
        cursor = None

        # Create orders sequentially using the same connection
        order_ids = []
        create_times = []

        for _ in range(num_orders):
            customer_id = random.choice(customers)
            items = pick_cart_items(products, max_items=max_items, max_qty=max_qty)
            order_id, elapsed_ms = create_order_with_items(conn, customer_id, items)
            order_ids.append(order_id)
            create_times.append(elapsed_ms)

        # Advance order statuses (sequential)
        for order_id in order_ids:
            advance_order_status(conn, order_id, "processing")

            if random.random() < cancel_rate:
                advance_order_status(conn, order_id, "cancelled")
                continue

            advance_order_status(conn, order_id, "shipped")
            advance_order_status(conn, order_id, "delivered")

        
        # Print timing statistics
        avg_time = sum(create_times) / len(create_times) if create_times else -1
        return avg_time
    except Exception:
        conn.rollback()
        return -1
    finally:
        if cursor is not None:
            cursor.close()
        conn.close()

def run_concurrent_simulations(num_runs=20):
    with ThreadPoolExecutor(max_workers=20) as executor:
        futures = [executor.submit(simulate_shopping_carts, num_orders=1000, max_items=10) for _ in range(num_runs)]
        results = [future.result() for future in futures]

    if results:
        valid_results = [r for r in results if r >= 0]
        if valid_results:
            grand_average = sum(valid_results) / len(valid_results)
            print(f"\n=== Grand Average Across Runs ===\nAverage time per order: {grand_average:.2f} ms")

run_concurrent_simulations(num_runs=100)

In [ ]:
# re-order low stock products
conn = connect_to_db(with_db=True)
cursor = conn.cursor()
try:
    conn.start_transaction()
    cursor.execute("UPDATE products SET stock_quantity = stock_quantity + 100 WHERE stock_quantity < 100")
    conn.commit()
    print("Re-ordered low stock products successfully.")
except Exception as e:
    conn.rollback()
    print(f"Error re-ordering low stock products: {e}")
finally:
    cursor.close()
    conn.close()